In [ ]:
import glob
import datasets
import json
import pandas as pd

In [ ]:
jsonl_files = glob.glob('data/0520orpo_model/*.jsonl')
len(jsonl_files)

In [ ]:
import random
from tqdm import tqdm
records=[]
done_tasks=[]
for file in jsonl_files:
    with open(file) as f:
        for line in tqdm(f):
            d=json.loads(line)
            if d["model_answer"]==d["ref_answer"]:
                d["model_answer"]=random.choice(("","分かりません","はい","いいえ"))
            
            task=d["question"]
            if task in done_tasks:
                continue
            d["data_source"]=d["dataset"]
            d.pop("dataset")
            records.append(d)
            done_tasks.append(task)

In [ ]:
records[0]

In [ ]:
parquet_path="data/0520orpo_model/1.parquet"
df=pd.DataFrame(records)
df.to_parquet(parquet_path,index=False)

In [ ]:
from huggingface_hub import HfApi, logging
hf = HfApi()
hf.upload_file(path_or_fileobj=parquet_path,
                path_in_repo=f"1.parquet",
                repo_id='hatakeyama-llm-team/AutoPreferenceDataset',
                  repo_type="dataset")




In [ ]:
#ds=datasets.Dataset.from_pandas(df)
#ds.push_to_hub('hatakeyama-llm-team/AutoPreferenceDataset')